In [96]:
from pathlib import Path
import sys
import subprocess
import json
import random
import importlib.util
import os

# --- clone repo if needed (Colab-friendly) ---
REPO_URL = "https://github.com/AhmedTarek62/wavesfm"
if (Path.cwd() / "main_finetune.py").exists():
    REPO_DIR = Path.cwd()
else:
    REPO_DIR = Path("/content/wavesfm")
    if not REPO_DIR.exists():
        subprocess.run(["git", "clone", REPO_URL, str(REPO_DIR)], check=True)
    os.chdir(REPO_DIR)

if str(REPO_DIR) not in sys.path:
    sys.path.insert(0, str(REPO_DIR))

# --- install missing deps (Colab) ---
def _ensure_pkg(module, pip_name=None):
    if importlib.util.find_spec(module) is None:
        pkg = pip_name or module
        subprocess.run([sys.executable, "-m", "pip", "install", "-q", pkg], check=True)

if importlib.util.find_spec("torch") is None:
    subprocess.run([
        sys.executable, "-m", "pip", "install", "-q",
        "torch", "torchvision", "torchaudio"
    ], check=True)

for mod, pip_name in [
    ("numpy", "numpy"),
    ("matplotlib", "matplotlib"),
    ("h5py", "h5py"),
    ("scipy", "scipy"),
    ("tqdm", "tqdm"),
    ("timm", "timm"),
    ("pandas", "pandas"),
    ("PIL", "pillow"),
    ("gdown", "gdown"),
]:
    _ensure_pkg(mod, pip_name)

import numpy as np
import matplotlib.pyplot as plt
import torch
import h5py


## Download raw datasets

This notebook supports a **subset of tasks** only. For deeper experiments (more tasks, configs, and training options), refer to the full repository: https://github.com/AhmedTarek62/wavesfm

### Vision tasks
- `sensing` — EfficientFi (Human Activity Sensing): https://github.com/xyanchen/WiFi-CSI-Sensing-Benchmark  
- `deepmimo-los` — DeepMIMO LoS/NLoS classification (generated locally via `preprocess_deepmimo.py`; no manual downloads)

### IQ tasks
- `rml` — RML 2022 (Modulation Classification): https://github.com/venkateshsathya/RML22  
- `uwb-industrial` — UWB Industrial Positioning: https://owncloud.fraunhofer.de/index.php/s/AXFjGY9IhswfBSa/download

#
> **Note:** Links were valid at the time this notebook was published. If a link breaks, please use the corresponding project page above to locate the latest download instructions.


In [106]:
# Configuration
TASK = "rml"  # {"sensing", "deepmimo-los", "rml", "uwb-industrial"}
DOWNLOAD_DATA = True      # set False if you already downloaded the raw files
SEED = 2
VAL_SPLIT = 0.2
STRATIFIED_TASKS = {"deepmimo-los"}
SMOOTH_TASKS = {"sensing": 0.1}

# Paths
DATA_ROOT = Path("data")
RAW_ROOT = DATA_ROOT / "raw"
CACHE_ROOT = DATA_ROOT / "cache"
OUTPUT_DIR = Path("runs/demo") / TASK
CHECKPOINT_DIR = Path("checkpoints")

for p in (RAW_ROOT, CACHE_ROOT, CHECKPOINT_DIR, OUTPUT_DIR):
    p.mkdir(parents=True, exist_ok=True)

# Raw dataset locations (after download/extraction)
HAS_DIR = RAW_ROOT / "NTU-Fi_HAR"              # EfficientFi HAS (sensing)
RML_DATA_FILE = RAW_ROOT / "rml2022"            # RML 2022 (rml)
UWB_INDUSTRIAL_DATA_FILE = RAW_ROOT / "industrial_training.pkl"  # UWB Industrial (uwb-industrial)
DEEPMIMO_SCENARIOS_DIR = RAW_ROOT / "deepmimo_scenarios"          # DeepMIMO scenarios clone target
DEEPMIMO_IMG_SIZE = 32
DEEPMIMO_CLONE = True


In [ ]:
# Optional dataset download

def run(cmd):
    # Run a shell command and fail loudly if it errors.
    subprocess.run(cmd, check=True)

if not DOWNLOAD_DATA:
    print("DOWNLOAD_DATA=False → assuming raw files already exist under:", RAW_ROOT)

elif TASK == "sensing":
    # EfficientFi HAS (Google Drive zip)
    HAS_URL = "https://drive.google.com/file/d/1DszE7byFzlpyI9gZvmVn51fTr8L1iZaI/view?usp=drive_link"
    zip_path = RAW_ROOT / "has.zip"
    run(["gdown", "--fuzzy", "-O", str(zip_path), HAS_URL])
    run(["unzip", "-o", str(zip_path), "-d", str(RAW_ROOT)])
    run(["rm", str(zip_path)])

elif TASK == "deepmimo-los":
    print("DeepMIMO is generated during preprocessing; no download step needed.")

elif TASK == "rml":
    # RML 2022 (Google Drive)
    RML_URL = "https://drive.google.com/file/d/1wrqnanHbmdFiP3DqaBjSVcBxoQ0nzD-a/view?usp=drive_link"
    out_path = RAW_ROOT / "rml2022"  # keep as downloaded filename; downstream code can point to it
    run(["gdown", "--fuzzy", "-O", str(out_path), RML_URL])

elif TASK == "uwb-industrial":
    # UWB Industrial Positioning (Fraunhofer ownCloud)
    UWB_URL = "https://owncloud.fraunhofer.de/index.php/s/AXFjGY9IhswfBSa/download"
    pkl_path = UWB_INDUSTRIAL_DATA_FILE
    run(["wget", "-O", str(pkl_path), UWB_URL])

else:
    raise ValueError(f"Unknown TASK={TASK!r}. Expected one of: sensing, deepmimo-los, rml, uwb-industrial.")


## Create preprocessed .h5 cache from raw data


In [ ]:
# --- cache path ---
if TASK == "sensing":
    CACHE_PATH = CACHE_ROOT / "has.h5"
elif TASK == "deepmimo-los":
    CACHE_PATH = CACHE_ROOT / "deepmimo.h5"
elif TASK == "rml":
    CACHE_PATH = CACHE_ROOT / "rml2022.h5"
elif TASK == "uwb-industrial":
    CACHE_PATH = CACHE_ROOT / "uwb-industrial.h5"
else:
    raise ValueError(f"Unsupported TASK: {TASK}")

print("Task:", TASK)
print("Cache path:", CACHE_PATH)

# --- preprocess (skip this cell if you already have CACHE_PATH) ---
if TASK == "sensing":
    subprocess.run([
        sys.executable, "preprocessing/preprocess_csi_sensing.py",
        "--data-path", str(HAS_DIR),
        "--output", str(CACHE_PATH),
        "--overwrite"], check=True)

elif TASK == "deepmimo-los":
    deepmimo_cmd = [
        sys.executable, "preprocessing/preprocess_deepmimo.py",
        "--output", str(CACHE_PATH),
        "--dataset-folder", str(DEEPMIMO_SCENARIOS_DIR),
        "--resize-size", str(DEEPMIMO_IMG_SIZE),
        "--overwrite",
    ]
    if DEEPMIMO_CLONE:
        deepmimo_cmd.append("--clone-scenarios")
    subprocess.run(deepmimo_cmd, check=True)

elif TASK == "rml":
    if not RML_DATA_FILE.exists():
        raise FileNotFoundError(f"Missing RML file at {RML_DATA_FILE}")
    subprocess.run([
        sys.executable, "preprocessing/preprocess_rml.py",
        "--data-file", str(RML_DATA_FILE),
        "--version", "2022",
        "--output", str(CACHE_PATH),
        "--overwrite",
    ], check=True)

elif TASK == "uwb-industrial":
    if not UWB_INDUSTRIAL_DATA_FILE.exists():
        raise FileNotFoundError(f"Missing UWB-Industrial file at {UWB_INDUSTRIAL_DATA_FILE}")
    subprocess.run([
        sys.executable, "preprocessing/preprocess_ipin_loc.py",
        "--data-path", str(UWB_INDUSTRIAL_DATA_FILE),
        "--output", str(CACHE_PATH),
        "--overwrite",
    ], check=True)


## Helpers


In [117]:
import re
from types import SimpleNamespace
from torch.utils.data import DataLoader, Subset

from data import build_datasets
from main_finetune import build_model


def _unwrap_subset_with_indices(ds):
    if not isinstance(ds, Subset):
        return ds, None
    indices = list(ds.indices)
    base = ds.dataset
    while isinstance(base, Subset):
        indices = [base.indices[i] for i in indices]
        base = base.dataset
    return base, np.asarray(indices, dtype=np.int64)


def _load_label_names(ds, task_info):
    base, _ = _unwrap_subset_with_indices(ds)
    labels = getattr(base, "labels", None)
    if labels:
        return list(labels)
    h5_path = getattr(base, "h5_path", None)
    if h5_path:
        with h5py.File(h5_path, "r") as h5:
            raw = h5.attrs.get("labels", None)
            if raw:
                return list(json.loads(raw))
            raw = h5.attrs.get("labels_los", None)
            if raw:
                return list(json.loads(raw))
    return [str(i) for i in range(task_info.num_outputs)]


def plot_samples(ds, task_info, *, seed=0, num_show=6):
    rng = random.Random(seed)
    num_show = min(num_show, len(ds))
    indices = rng.sample(range(len(ds)), num_show)

    def _to_numpy(x):
        if torch.is_tensor(x):
            return x.detach().cpu().numpy()
        return np.asarray(x)

    def _label_text(label):
        if torch.is_tensor(label):
            label = label.detach().cpu().numpy()
        label = np.asarray(label)
        if label.shape == ():
            return f"label={int(label)}"
        return f"label={np.array2string(label, precision=2, separator=',')}"

    def _plot_iq(ax, x):
        # x expected as (2, L) or (2, A, L) or similar
        if x.ndim >= 3:
            i = np.asarray(x[0]).reshape(x.shape[1], -1)[0]
            q = np.asarray(x[1]).reshape(x.shape[1], -1)[0]
        elif x.ndim == 2:
            i = x[0]
            q = x[1]
        else:
            flat = x.reshape(-1)
            half = flat.size // 2
            i = flat[:half]
            q = flat[half:]
        ax.plot(i, label="I")
        ax.plot(q, label="Q")
        ax.legend(fontsize=6)
        ax.axis("off")

    def _plot_sample(ax, sample, label, modality):
        x = _to_numpy(sample)
        if modality in ("iq", "cir"):
            _plot_iq(ax, x)
        else:
            if x.ndim == 3 and x.shape[0] in (1, 2, 3):
                img = np.moveaxis(x, 0, -1)
                if img.shape[-1] == 1:
                    ax.imshow(img[..., 0], cmap="viridis")
                elif img.shape[-1] == 2:
                    mag = np.sqrt(img[..., 0] ** 2 + img[..., 1] ** 2)
                    ax.imshow(mag, cmap="viridis")
                else:
                    ax.imshow(img)
                ax.axis("off")
            elif x.ndim == 2:
                ax.imshow(x, aspect="auto", cmap="viridis")
                ax.axis("off")
            else:
                ax.plot(x.flatten())
                ax.axis("off")
        ax.set_title(_label_text(label))

    fig, axes = plt.subplots(1, num_show, figsize=(3 * num_show, 5))
    if num_show == 1:
        axes = [axes]
    for ax, idx in zip(axes, indices):
        sample, label = ds[idx]
        _plot_sample(ax, sample, label, task_info.modality)

    fig.suptitle(f"{TASK} samples (train split)")
    plt.show()


def pick_eval_ckpt(output_dir: Path) -> Path:
    best = output_dir / "best.pth"
    if best.exists():
        return best
    candidates = list(output_dir.glob("checkpoint_*.pth"))
    if candidates:
        def _ckpt_epoch(path: Path) -> int:
            match = re.search(r"checkpoint_(\d+)\.pth", path.name)
            return int(match.group(1)) if match else -1
        return max(candidates, key=_ckpt_epoch)
    raise FileNotFoundError(f"No checkpoints found in {output_dir}")


def load_eval_model(ckpt_path: Path, task_info, *, device: str = "cpu"):
    model_args = SimpleNamespace(
        model="vit_multi_small",
        global_pool="token",
        vis_patch=16,
        vis_img_size=DEEPMIMO_IMG_SIZE if TASK.startswith("deepmimo") else 224,
        iq_segment_len=16,
        iq_downsample=None,
        iq_target_len=256,
        use_conditional_ln=True,
    )
    model = build_model(model_args, task_info)
    ckpt = torch.load(ckpt_path, map_location="cpu", weights_only=False)
    state = ckpt.get("model", ckpt) if isinstance(ckpt, dict) else ckpt
    model.load_state_dict(state, strict=False)
    model.to(device)
    model.eval()
    return model


def plot_confusion_matrix(model, val_ds, task_info, *, batch_size=256, device="cpu", annotate=True):
    loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False, num_workers=2)
    num_classes = task_info.num_outputs
    conf = np.zeros((num_classes, num_classes), dtype=int)

    with torch.no_grad():
        for batch in loader:
            samples, targets = batch[0].to(device), batch[1].to(device).long()
            outputs = model(samples)
            preds = outputs.argmax(dim=1)
            for t, p in zip(targets.cpu().numpy(), preds.cpu().numpy()):
                conf[int(t), int(p)] += 1

    with np.errstate(divide="ignore", invalid="ignore"):
        row_sums = conf.sum(axis=1, keepdims=True)
        conf_norm = np.divide(conf, row_sums, out=np.zeros_like(conf, dtype=float), where=row_sums != 0)

    label_names = _load_label_names(val_ds, task_info)
    fig, ax = plt.subplots(figsize=(6, 6))
    im = ax.imshow(conf_norm, cmap="Blues", vmin=0.0, vmax=1.0)
    ax.set_title(f"Confusion Matrix ({TASK})")
    ax.set_xlabel("Predicted")
    ax.set_ylabel("True")
    ax.set_xticks(range(num_classes))
    ax.set_yticks(range(num_classes))
    ax.set_xticklabels(label_names, rotation=45, ha="right")
    ax.set_yticklabels(label_names)

    if annotate:
        for i in range(num_classes):
            for j in range(num_classes):
                val = conf_norm[i, j]
                ax.text(j, i, f"{val:.2f}", ha="center", va="center", color="black", fontsize=8)

    fig.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    plt.tight_layout()
    plt.show()


def plot_uwb_position_density(ds, *, max_points=5000, bins=60):
    base, idxs = _unwrap_subset_with_indices(ds)
    if not hasattr(base, "loc_min") or not hasattr(base, "loc_max"):
        print("UWB location metadata missing; skipping density plot.")
        return

    total = len(ds)
    if total == 0:
        print("No samples available for density plot.")
        return
    rng = np.random.default_rng(SEED)
    if total > max_points:
        sample_indices = rng.choice(total, size=max_points, replace=False)
    else:
        sample_indices = np.arange(total)

    coords = []
    for i in sample_indices:
        _, loc = ds[int(i)]
        if torch.is_tensor(loc):
            loc = loc.detach().cpu().numpy()
        loc = np.asarray(loc).reshape(-1)[:2]
        coords.append(loc)

    coords = np.asarray(coords, dtype=np.float32)
    loc_min = np.asarray(base.loc_min[:2].cpu(), dtype=np.float32)
    loc_max = np.asarray(base.loc_max[:2].cpu(), dtype=np.float32)
    coords = (coords + 1.0) * 0.5 * (loc_max - loc_min) + loc_min

    fig, ax = plt.subplots(figsize=(6, 5))
    ax.hist2d(coords[:, 0], coords[:, 1], bins=bins, cmap="magma")
    ax.set_title("UWB-Industrial Position Density")
    ax.set_xlabel("X")
    ax.set_ylabel("Y")
    plt.tight_layout()
    plt.show()


def plot_rml_accuracy_vs_snr(model, val_ds, *, batch_size=256, device="cpu"):
    base, idxs = _unwrap_subset_with_indices(val_ds)
    snr_by_index = getattr(base, "snr_by_index", None)
    if snr_by_index is None:
        print("SNR metadata not found; skipping accuracy vs SNR plot.")
        return

    snrs = np.asarray(snr_by_index, dtype=np.int16)
    if idxs is not None:
        snrs = snrs[idxs]

    loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False, num_workers=2)
    offset = 0
    correct = {}
    total = {}

    with torch.no_grad():
        for batch in loader:
            samples, targets = batch[0].to(device), batch[1].to(device).long()
            outputs = model(samples)
            preds = outputs.argmax(dim=1)
            batch_snrs = snrs[offset: offset + len(targets)]
            offset += len(targets)

            for snr_val, p, t in zip(batch_snrs, preds.cpu().numpy(), targets.cpu().numpy()):
                snr_val = int(snr_val)
                total[snr_val] = total.get(snr_val, 0) + 1
                correct[snr_val] = correct.get(snr_val, 0) + int(p == t)

    snr_levels = sorted(total.keys())
    acc = [correct[s] / total[s] for s in snr_levels]

    fig, ax = plt.subplots(figsize=(6, 4))
    ax.plot(snr_levels, acc, marker="o")
    ax.set_title("RML Accuracy vs SNR")
    ax.set_xlabel("SNR")
    ax.set_ylabel("Accuracy")
    ax.set_ylim(0.0, 1.0)
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()


## Visualize cached samples


In [ ]:
train_ds, val_ds, task_info = build_datasets(
    TASK,
    str(CACHE_PATH),
    val_path=None,
    val_split=VAL_SPLIT,
    stratified_split=TASK in STRATIFIED_TASKS,
    seed=SEED,
)

plot_samples(train_ds, task_info, seed=SEED)
print("Task info:", task_info.modality)

## Download pretrained checkpoint


In [ ]:
from hub import download_pretrained

HF_REPO = "ahmedaboulfo/wavesfm"
HF_FILE = "wavesfm-v1p0.pth"

PRETRAINED_PATH = Path(
    download_pretrained(repo_id=HF_REPO, filename=HF_FILE, cache_dir=str(CHECKPOINT_DIR))
)
print("Downloaded to:", PRETRAINED_PATH)


## Finetune


In [ ]:
# --- finetune (match run_finetune_all.py defaults, seed 0) ---
EPOCHS_BY_TASK = {"rml": 50,}
DEFAULT_EPOCHS = 100
BATCH_SIZE_BY_TASK = {"rml": 2048,}
DEFAULT_BATCH = 256

epochs = EPOCHS_BY_TASK.get(TASK, DEFAULT_EPOCHS)
batch_size = BATCH_SIZE_BY_TASK.get(TASK, DEFAULT_BATCH)

train_cmd = [
    sys.executable, "main_finetune.py",
    "--task", TASK,
    "--train-data", str(CACHE_PATH),
    "--output-dir", str(OUTPUT_DIR),
    "--batch-size", str(batch_size),
    "--num-workers", "2",
    "--epochs", str(epochs),
    "--seed", str(SEED),
    "--val-split", str(VAL_SPLIT),
    "--model", "vit_multi_small",
    "--warmup-epochs", "5",
    "--use-conditional-ln",
    "--finetune", str(PRETRAINED_PATH),
]

if TASK in STRATIFIED_TASKS:
    train_cmd += ["--stratified-split", "--class-weights"]
if TASK in SMOOTH_TASKS:
    train_cmd += ["--smoothing", str(SMOOTH_TASKS[TASK])]
if TASK.startswith("deepmimo"):
    train_cmd += ["--vis-img-size", str(DEEPMIMO_IMG_SIZE)]

subprocess.run(train_cmd, check=True)


## Load evaluation checkpoint


In [ ]:
best_ckpt = pick_eval_ckpt(OUTPUT_DIR)
print(f"Eval checkpoint: {best_ckpt}")

device = "cuda" if torch.cuda.is_available() else "cpu"
train_ds, val_ds, task_info = build_datasets(
    TASK,
    str(CACHE_PATH),
    val_path=None,
    val_split=VAL_SPLIT,
    stratified_split=TASK in STRATIFIED_TASKS,
    seed=SEED,
)

eval_model = load_eval_model(best_ckpt, task_info, device=device)


## Evaluation & plots


In [ ]:
# --- eval-only metrics ---
eval_cmd = [
    sys.executable, "main_finetune.py",
    "--task", TASK,
    "--train-data", str(CACHE_PATH),
    "--batch-size", str(batch_size),
    "--num-workers", "2",
    "--seed", str(SEED),
    "--val-split", str(VAL_SPLIT),
    "--model", "vit_multi_small",
    "--warmup-epochs", "5",
    "--use-conditional-ln",
    "--eval-only",
    "--finetune", str(best_ckpt),
]

if TASK in STRATIFIED_TASKS:
    eval_cmd += ["--stratified-split", "--class-weights"]
if TASK in SMOOTH_TASKS:
    eval_cmd += ["--smoothing", str(SMOOTH_TASKS[TASK])]
if TASK.startswith("deepmimo"):
    eval_cmd += ["--vis-img-size", str(DEEPMIMO_IMG_SIZE)]

subprocess.run(eval_cmd, check=True)

# --- plots ---
if task_info.target_type == "classification":
    plot_confusion_matrix(eval_model, val_ds, task_info, batch_size=batch_size, device=device, annotate=(TASK != "rml"))

if TASK == "uwb-industrial":
    plot_uwb_position_density(val_ds)

if TASK == "rml":
    plot_rml_accuracy_vs_snr(eval_model, val_ds, batch_size=batch_size, device=device)
